# Spectral Line Search

This example shows how to use dysh to search for spectral lines.
Searches can be done for a user-specified frequency range or for the frequency axis of a Spectrum.
You can use a remote query to [Splatalogue](https://splatalogue.online/) or use local tables distributed with dysh.

First we show how to do a search using a [``SpectralLineSearch``](https://dysh.readthedocs.io/en/latest/reference/modules/dysh.line.html#dysh.line.SpectralLineSearchClass) object, then using a [``Spectrum``](https://dysh.readthedocs.io/en/latest/reference/modules/dysh.spectra.html#dysh.spectra.spectrum.Spectrum) object. Examples are given both with and without a redshift value. 

## Loading Modules
We start by loading the modules we will use. 

In [ ]:
# These modules are required for this example.
from dysh.line import SpectralLineSearch
from dysh.spectra import Spectrum
from astropy import units as u

## Example 1.  Doing an online or local search with SpectralLineSearch
[``SpectralLineSearch``](https://dysh.readthedocs.io/en/latest/reference/modules/dysh.line.html#dysh.line.SpectralLineSearchClass) is a thin wrapper on top of [``astroquery.splatalogue.Splatalogue``](https://astroquery.readthedocs.io/en/latest/splatalogue/splatalogue.html), with some additional conveniences for working in dysh and with GBT data.
You search a catalog in a given frequency range, optionally narrow down to specific molecules, line strengths, energies, etc.
The return object is an astropy [``Table``](https://docs.astropy.org/en/stable/api/astropy.table.Table.html#astropy.table.Table).
Below are a few example searches.

### Do an online search for N2H+ lines between 80 and 400 GHz
Online searching of Splatalogue is the default. 
Note in this example we use an initial space in the ``chemical_name`` to eliminate other molecules that have the string ``'N2H+'`` in them. 
We set an ``intensity_lower_limit`` to weed out some of the weaker satellite lines.
When giving an ``intensity_lower_limit``, one must also provide an ``intensity_type`` string, a minimum match to ``'CDMS/JPL (log)'``, ``'Sij-mu2'``, or ``'Aij (log)'``. 

In [ ]:
minfreq = 80*u.GHz
maxfreq = 400*u.GHz
table = SpectralLineSearch.query_lines(minfreq, maxfreq, 
                                       chemical_name=" N2H+", 
                                       intensity_lower_limit=-3, 
                                       intensity_type="CDMS")

In [ ]:
print(f"{len(table)} rows returned.")

To inspect the available columns use the [``Table.colnames``](https://docs.astropy.org/en/stable/api/astropy.table.Table.html#astropy.table.Table.colnames) attribute:

In [ ]:
print(table.colnames)

The ``rest_frequency`` column is the rest frequency of the transition in MHz. 

Some columns in the table often have embedded HTML.
That's the way they come from Splatalogue.
`¯\_(ツ)_/¯`



In [ ]:
table

### Do a local search for methyl formate.
This uses a GBT-specific catalog distributed with dysh.
You can also provide your own catalog in an astropy [``Table``](https://docs.astropy.org/en/stable/api/astropy.table.Table.html#astropy.table.Table) format.
The first time you use a given catalog, there will be some additional overhead to read it in and cache it.
Subsequent searches will be faster because they use the cached version.
Note the default regular expression matching is case-insensitive. 
The ``gbtlines`` catalog has most lines from 300 MHz to 120 GHz, and commonly observed redshifted lines from 120 GHz to 5 THz.

In [ ]:
minfreq = 400*u.MHz
maxfreq = 5550*u.MHz
table = SpectralLineSearch.query_lines(minfreq, maxfreq,
                                       cat='gbtlines', 
                                       chemical_name="methyl formate")

In [ ]:
print(f"{len(table)} rows returned.")

In [ ]:
table

### Search for Recombination Lines
dysh has special methods to search for recombination lines of Hydrogen, Helium, and Carbon. 
[``SpectralLineSearch.recomb``](https://dysh.readthedocs.io/en/latest/reference/modules/dysh.line.html#dysh.line.SpectralLineSearchClass.recomb) will search for a specific atom/transition, while [``SpectralLineSearch.recomball``](https://dysh.readthedocs.io/en/latest/reference/modules/dysh.line.html#dysh.line.SpectralLineSearchClass.recomball) will search for all three species. 

In [ ]:
minfreq = 300*u.MHz
maxfreq = 2.0*u.GHz
table = SpectralLineSearch.recomb(minfreq, maxfreq,
                                  cat='gbtrecomb',
                                  line="Hebeta")

In [ ]:
table

Search all recombination lines.
As with any astropy [``Table``](https://docs.astropy.org/en/stable/api/astropy.table.Table.html#astropy.table.Table), you can limit the displayed columns with a list.  Alternatively, you could limit the columns returns with the parameter `columns=['name', 'chemical_name', 'rest_frequency']`.

In [ ]:
table = SpectralLineSearch.recomball(minfreq, maxfreq,
                                     cat='gbtrecomb')
table[['name', 'chemical_name', 'rest_frequency']]

## Add in a redshift
Suppose you wanted to find what CO lines would be in your observing band if they were emitted at a redshift of 1.2? You can add the `redshift` parameter to the search.  The minimum and maximum frequency still refer to the observed band.  In the resulting table, ``obs_frequency`` is the redshifted frequency value, that is the frequency at which the line will appear.

In [ ]:
min_frequency=88*u.GHz
max_frequency=120*u.GHz
redshift=1.2
table = SpectralLineSearch.query_lines(min_frequency, max_frequency,cat='gbtlines', 
                                       redshift=redshift, chemical_name="Carbon Monoxide",
                                       intensity_lower_limit=-5,intensity_type="cdms")

In [ ]:
table[['name', 'chemical_name', 'rest_frequency', 'obs_frequency',]]

## Example 2. Search within the frequency range of a Spectrum
The [``Spectrum.query_lines``](https://dysh.readthedocs.io/en/latest/reference/modules/dysh.spectra.html#dysh.spectra.spectrum.Spectrum.query_lines). [``Spectrum.recomb``](https://dysh.readthedocs.io/en/latest/reference/modules/dysh.spectra.html#dysh.spectra.spectrum.Spectrum.recomb), and [``Spectrum.recomball``](https://dysh.readthedocs.io/en/latest/reference/modules/dysh.spectra.html#dysh.spectra.spectrum.Spectrum.recomball) methods of [``Spectrum``](https://dysh.readthedocs.io/en/latest/reference/modules/dysh.spectra.html#dysh.spectra.spectrum.Spectrum) will use the minimum and maximum frequencies of the spectral axis and the `redshift` attribute of the Spectrum.
Other keywords are the same as in [``SpectralLineSearch``](https://dysh.readthedocs.io/en/latest/reference/modules/dysh.line.html#dysh.line.SpectralLineSearchClass).

For this example we construct a fake spectrum, with 16384 channels that are 1 kHz wide and a rest frequency of 1.4240575 GHz.

In [ ]:
cdelt1 = float((1000*u.Hz).value)
nchan = 16384
restfreq = float((1.4240575*u.GHz).to(u.Hz).value)
crval1 = restfreq
s = Spectrum.fake_spectrum(nchan=nchan, 
                           crval1=crval1, 
                           restfrq=restfreq, 
                           freqres=cdelt1, 
                           cdelt1=cdelt1, 
                           obsfreq=restfreq, 
                           dopfreq=restfreq, 
                           object='MyFakeSrc')

In [ ]:
print(f"Spectrum redshift is {s.redshift}")

In [ ]:
s.plot(vel_frame='lsrk', doppler_convention='radio', xaxis_unit='km/s')

Find the hydrogen recombination lines that could be in this spectrum, and display their name, chemical name and frequency (in MHz).

In [ ]:
s.recomb(line='hydrogen', cat='gbtrecomb', columns=['name', 'chemical_name', 'rest_frequency','obs_frequency'])

Find all lines in the spectrum's frequency range.

In [ ]:
s.query_lines(cat='gbtlines',columns=['name', 'chemical_name', 'rest_frequency','obs_frequency'])

To clear the cache, use:

In [ ]:
SpectralLineSearch.clear_cache()